# Calculating normals from height map

I want to figure out how to calculate normals for height map values generated by $z=\sin(x) \cdot \sin(y)$ for $x,y \in [0, \pi]$ equation.

Let's assume folowing program which generates $512x512$ array of $z$ values.

In [2]:
import numpy as np

image_width = 512
image_height = 512

# Step 1: Generate the 2D array
x = np.linspace(0, np.pi, image_width)
y = np.linspace(0, np.pi, image_height)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.sin(Y)

In [4]:
print(f'Z.shape={Z.shape}')  #= (rows, columns)
print(f'Z.min={np.min(Z)}, Z.max={np.max(Z)}')

Z.shape=(512, 512)
Z.min=0.0, Z.max=0.9999905507663935


and a $x_0$ and $y_0$ uv (array/texture) coordinates we want to calculate normal, defined as

In [14]:
x0=70
y0=50

In [15]:
def print_window(arr, x, y, width=3):
	half = width // 2
	col_beg = max(0, x - half)
	col_end = min(arr.shape[1], x + half+1)
	row_beg = max(0, y - half)
	row_end = min(arr.shape[0], y + half+1)
	win = arr[row_beg:row_end, col_beg:col_end]
	print(win)

# print 3x3 window with a center in p point?
p=(y0,x0)
Zp=Z[y0, x0]  # row-major order, which means the first index corresponds to the row and the second index corresponds to the column
print(f'h{p}={Zp}')
print_window(Z, x0, y0)

h(50, 70)=0.12623370786336227
[[0.12212652 0.12378669 0.12544218]
 [0.12454072 0.12623371 0.12792192]
 [0.12695022 0.12867595 0.13039683]]


In [16]:
# 'uv' mapping in one dimension
N=5
a=np.linspace(0,1,N)
print(a)

n=2  # get value of third element
1/(N-1)*n

[0.   0.25 0.5  0.75 1.  ]


0.5

Following code transforms from uv space to function space.
> **note**: function space means spece where z=sin(x)*sin(y) for x,y in [0, pi] values was calculated.

In [17]:
x0_word=np.pi/(image_width-1)*x0
y0_word=np.pi/(image_height-1)*y0
p=np.array([x0_word, y0_word, Z[y0, x0]])
print(f'p={p}')

p=[0.43035516 0.30739654 0.12623371]


We can now take four adjacent point and calculate normal from them, this way

In [18]:
def normalize(v):
	norm = np.linalg.norm(v)
	return v / norm if norm != 0 else v

def to_word(x, y, z):
	x_w = np.pi/(image_width-1)*x
	y_w = np.pi/(image_height-1)*y
	return np.array([x_w, y_w, z])

p0 = to_word(x0-1, y0, Z[y0, x0-1])
p1 = to_word(x0+1, y0, Z[y0, x0+1])
p2 = to_word(x0, y0-1, Z[y0-1, x0])
p3 = to_word(x0, y0+1, Z[y0+1, x0])
dx = p1 - p0
dy = p3 - p2
n_height = normalize(np.cross(dx, dy))
print(f'n_height={n_height}')

n_height=[-0.24757194 -0.35799279  0.90030511]


Now let's calculate normal by a function partial derivatives

In [19]:
p_xy = to_word(x0, y0, Z[y0,x0])
cos_xy = np.cos(p_xy[0:2])
sin_xy = np.sin(p_xy[0:2])
dF = np.array([cos_xy[0]*sin_xy[1], sin_xy[0]*cos_xy[1], -1])
n = normalize(-dF)
print(f'n={n}')

n=[-0.24757321 -0.35799462  0.90030404]


Now compare both results

In [21]:
np.allclose(n, n_height)

True

Computing from 8bit height map gives slightly different results due to loss of precision.